## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Mix Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import seaborn as sns 
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#### <font color='blue'>Libraries for Critical Diagram 

In [7]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
from Orange.evaluation import compute_CD, graph_ranks
from sklearn.metrics import mean_absolute_error

### <font color='green'> Load Dataset 

In [8]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [9]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [10]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [11]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_std,skinTemperature_var,heartRate_std,heartRate_var,sao2_std,bpCuffMean_std,sao2_var,...,BIRTH_WEIGHT,bpCuffSystolic_mean,bpCuffDiastolic_mean,sao2_mean,temperature_std,glucometer_mean,temperature_var,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.134078,0.017977,0.22911,0.057937,0.097817,0.230854,0.009568,...,0.264267,0.602649,0.457143,0.881455,0.125281,0.312593,0.015695,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.086672,0.007512,0.28258,0.086102,0.080938,0.142539,0.006551,...,0.437819,0.644907,0.479330,0.926056,0.061399,0.272551,0.003770,0.351928,0.392157,0.196784


In [12]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Mix ML) - (+FS, +HO)

In [13]:
def get_individual_result(model, dsel_x, dsel_y, test_x, test_y): 
    model.fit(dsel_x, dsel_y)
    y_preds = model.predict(test_x) 
    yproba = model.predict_proba(test_x)[::,1] 
    
    acc = accuracy_score(test_y, y_preds)
    prec = precision_score(test_y, y_preds)
    rec = recall_score(test_y, y_preds)
    f1 = f1_score(test_y, y_preds)
    fpr, tpr, _ = roc_curve(test_y,  yproba) 
    auc = roc_auc_score(test_y, yproba)
    
    return {"acc": acc, "prec": prec, "rec": rec, "f1":f1, "fpr": fpr, "tpr":tpr, "auc": auc}
    

In [14]:
def hold_out_mix_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    
    model_svc = SVC(kernel='linear', C=0.007, gamma=0.2, degree=3, probability=True, class_weight='balanced')
#     model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=3) # depth was 3  
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    
    model_rf  = RandomForestClassifier(criterion='gini', n_estimators=100, max_depth=3)
    model_cat = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
#     model_lgb = LGBMClassifier(max_depth=1, n_estimators=150, objective="binary")
    model_ada = AdaBoostClassifier(n_estimators=250, learning_rate=0.01)
    
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_rf.fit(X_train, y_train_en) 
    model_cat.fit(X_train, y_train_en) 
#     model_lgb.fit(X_train, y_train_en)
    model_ada.fit(X_train, y_train_en)
    
    classifiers_names = ["MLP", "Logistic Regression", "Random Forest", 
                         "CatBoost", "AdaBoost"] # "Decision Tree",
    # "AdaBoost" , "LGBM"
    pool_classifiers = [model_ml_perceptron, 
                        model_lr, 
#                         model_svc, 
                        model_rf, 
                        model_cat,
                        model_ada] 
    c_acc_list = [] 
    
    for cls in pool_classifiers:
        y_preds_c = cls.predict(X_test_mm) 
        c_acc_list.append(accuracy_score(y_test, y_preds_c))           
        
        
    classifiers_results =  {'name': classifiers_names, 
                            'accuracy': c_acc_list}
    
    clsDF = pd.DataFrame.from_dict(classifiers_results)
        
    # DES STARTS
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=7) 
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9)
    fire_metades = METADES(pool_classifiers, DFP=True, k=9) 
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=7) # 7 was 96%
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        results_dct = get_individual_result(e_cls, X_dsel, y_dsel, X_test_mm, y_test)
        acc_list.append(results_dct["acc"])
        precision_list.append(results_dct["prec"])
        recall_list.append(results_dct["rec"]) 
        f1_lists.append(results_dct["f1"]) 
        auc_list.append(results_dct["auc"])
        fpr_list.append(results_dct["fpr"])
        tpr_list.append(results_dct["tpr"])

    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df, clsDF

In [15]:
results_data = []
classifier_results_data = []
for i in range(0, 15):
    result, cls_results = hold_out_mix_ML()
    results_data.append(result)
    classifier_results_data.append(cls_results)
    

fireResultsDF = pd.concat(results_data)
classifiersResultsDF = pd.concat(classifier_results_data)

Iteration 1, loss = 0.65161448
Iteration 2, loss = 0.53385789
Iteration 3, loss = 0.44910936
Iteration 4, loss = 0.40074651
Iteration 5, loss = 0.36816005
Iteration 6, loss = 0.34609957
Iteration 7, loss = 0.33003940
Iteration 8, loss = 0.31555330
Iteration 9, loss = 0.30619362
Iteration 10, loss = 0.29958715
Iteration 11, loss = 0.28731648
Iteration 1, loss = 0.63554610
Iteration 2, loss = 0.52807818
Iteration 3, loss = 0.44764027
Iteration 4, loss = 0.39474021
Iteration 5, loss = 0.36411297
Iteration 6, loss = 0.34622784
Iteration 7, loss = 0.33659109
Iteration 8, loss = 0.32306084
Iteration 9, loss = 0.31542161
Iteration 10, loss = 0.30354256
Iteration 11, loss = 0.29150450
Iteration 1, loss = 0.64415434
Iteration 2, loss = 0.53167710
Iteration 3, loss = 0.45042641
Iteration 4, loss = 0.39878711
Iteration 5, loss = 0.36560365
Iteration 6, loss = 0.34359528
Iteration 7, loss = 0.32638478
Iteration 8, loss = 0.31126279
Iteration 9, loss = 0.30025591
Iteration 10, loss = 0.29296380
Ite

Iteration 1, loss = 0.62212518
Iteration 2, loss = 0.49540100
Iteration 3, loss = 0.41848159
Iteration 4, loss = 0.37248772
Iteration 5, loss = 0.34439676
Iteration 6, loss = 0.32656736
Iteration 7, loss = 0.31241807
Iteration 8, loss = 0.30081813
Iteration 9, loss = 0.28838222
Iteration 10, loss = 0.27961137
Iteration 11, loss = 0.27078580
Iteration 1, loss = 0.68490249
Iteration 2, loss = 0.57282882
Iteration 3, loss = 0.48214950
Iteration 4, loss = 0.41521227
Iteration 5, loss = 0.37521527
Iteration 6, loss = 0.35176187
Iteration 7, loss = 0.33258877
Iteration 8, loss = 0.31604372
Iteration 9, loss = 0.30938384
Iteration 10, loss = 0.29728300
Iteration 11, loss = 0.28905858
Iteration 1, loss = 0.63601039
Iteration 2, loss = 0.52505741
Iteration 3, loss = 0.44444496
Iteration 4, loss = 0.39374949
Iteration 5, loss = 0.36023410
Iteration 6, loss = 0.33977701
Iteration 7, loss = 0.32403620
Iteration 8, loss = 0.31155239
Iteration 9, loss = 0.29800875
Iteration 10, loss = 0.28915606
Ite

In [16]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [17]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.958,0.004,0.923,0.007,1.000,0.000,0.960,0.004,0.985,0.002
1,KNORA-U,0.957,0.004,0.920,0.007,1.000,0.000,0.959,0.004,0.984,0.002
2,FIRE-KNORA-E,0.965,0.003,0.935,0.005,1.000,0.000,0.967,0.003,0.988,0.001
3,KNORA-E,0.965,0.003,0.935,0.006,1.000,0.000,0.967,0.003,0.988,0.001
4,FIRE-METADES,0.943,0.020,0.940,0.004,0.947,0.040,0.943,0.021,0.987,0.001
5,METADES,0.937,0.014,0.941,0.005,0.934,0.029,0.937,0.015,0.987,0.001
6,FIRE-DESKNN,0.946,0.012,0.949,0.004,0.944,0.023,0.946,0.012,0.988,0.001
7,DESKNN,0.946,0.012,0.949,0.005,0.944,0.023,0.946,0.012,0.987,0.001
8,FIRE-MCB,0.955,0.003,0.934,0.006,0.980,0.004,0.956,0.003,0.987,0.001
9,MCB,0.955,0.004,0.934,0.006,0.980,0.004,0.956,0.004,0.986,0.001


In [22]:
finalResultsDF.to_csv("Results/mix_pool_results/des_mix_pool_5_cls_results_2-3.csv", index=False)

In [19]:
classifiers_names = ["MLP", "Logistic Regression", "Random Forest", "CatBoost", "AdaBoost"] 
# "AdaBoost", "LGBM"

accuracy_list = []
accuracy_std_list = []
for n in classifiers_names:
    accuracy_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.mean())
    accuracy_std_list.append(classifiersResultsDF[classifiersResultsDF.name == n].accuracy.std())

final_cls_results = {"classifier": classifiers_names, 
                     "accuracy": accuracy_list, 
                     "accuracy_std": accuracy_std_list}

finalClassifierResultsDF = pd.DataFrame.from_dict(final_cls_results)

In [20]:
finalClassifierResultsDF

,classifier,accuracy,accuracy_std
0,MLP,0.944121,1.220771e-02
1,Logistic Regression,0.941748,4.596760e-16
2,Random Forest,0.938655,1.598310e-03
3,CatBoost,0.941388,1.363404e-02
4,AdaBoost,0.950378,2.298380e-16


In [21]:
finalResultsDF.describe().T

,count,mean,std,min,25%,50%,75%,max
accuracy,14.0,0.951500,0.009936,0.937,0.94375,0.9550,0.95800,0.965
accuracy_std,14.0,0.008429,0.005932,0.003,0.00400,0.0045,0.01350,0.020
precision,14.0,0.934786,0.009150,0.920,0.92800,0.9350,0.94075,0.949
precision_std,14.0,0.005643,0.001151,0.004,0.00500,0.0055,0.00600,0.008
recall,14.0,0.971500,0.029175,0.934,0.94400,0.9800,1.00000,1.000
recall_std,14.0,0.013214,0.015232,0.000,0.00000,0.0040,0.02750,0.040
f1_score,14.0,0.952429,0.010938,0.937,0.94375,0.9560,0.96000,0.967
f1_std,14.0,0.008571,0.006442,0.003,0.00325,0.0040,0.01425,0.021
auc,14.0,0.986500,0.001506,0.983,0.98625,0.9870,0.98700,0.988
auc_std,14.0,0.001286,0.000611,0.001,0.00100,0.0010,0.00100,0.003
